## Imports

In [1]:
import keras

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [84]:
import pandas as pd
import wget
import pickle
import collections
import os,sys

## Paths and variables

In [97]:
vocab_file = 'vocabulary.csv'
cleaned_data_file = 'cleaned_jokes.csv'
seq_length = 30
sequences_step = 1

## data cleaning

In [10]:
filename = wget.download('https://github.com/amoudgl/short-jokes-dataset/raw/master/shortjokes.csv')

In [11]:
print(filename)

shortjokes (1).csv


In [16]:
joke_text = []
joke_id = []
num_jokes = 0
with open(filename,"r") as fip:
    jokes = fip.read().split("\n")
    for j in jokes[:10]:
        print(j)
    num_jokes = len(jokes)
    for i,joke in enumerate(jokes):
        if i == 0:
            continue
        temp = joke.split(',')
        if len(temp) < 2:
            continue
        joke_text.append(temp[1])
        joke_id.append(temp[0])

"ID","Joke"
1,"[me narrating a documentary about narrators] ""I can't hear what they're saying cuz I'm talking"""
2,"Telling my daughter garlic is good for you. Good immune system and keeps pests away.Ticks, mosquitos, vampires... men."
3,"I've been going through a really rough period at work this week It's my own fault for swapping my tampax for sand paper."
4,"If I could have dinner with anyone, dead or alive... ...I would choose alive. -B.J. Novak-"
5,"Two guys walk into a bar. The third guy ducks."
6,"Why can't Barbie get pregnant? Because Ken comes in a different box. Heyooooooo"
7,"Why was the musician arrested? He got in treble."
8,"Did you hear about the guy who blew his entire lottery winnings on a limousine? He had nothing left to chauffeur it."
9,"What do you do if a bird shits on your car? Don't ask her out again."


In [17]:
num_jokes

231659

In [56]:
joke_text[:10]

['"[me narrating a documentary about narrators] ""I can\'t hear what they\'re saying cuz I\'m talking"""',
 '"Telling my daughter garlic is good for you. Good immune system and keeps pests away.Ticks',
 '"I\'ve been going through a really rough period at work this week It\'s my own fault for swapping my tampax for sand paper."',
 '"If I could have dinner with anyone',
 '"Two guys walk into a bar. The third guy ducks."',
 '"Why can\'t Barbie get pregnant? Because Ken comes in a different box. Heyooooooo"',
 '"Why was the musician arrested? He got in treble."',
 '"Did you hear about the guy who blew his entire lottery winnings on a limousine? He had nothing left to chauffeur it."',
 '"What do you do if a bird shits on your car? Don\'t ask her out again."',
 '"He was a real gentlemen and always opened the fridge door for me"']

In [57]:
def cleanData(raw_data):
    processed_data = []
    for d in raw_data:
        d = d.strip().lower()
        d = d.replace('.',' EOS ')
        d  = [x for x in d if (ord(x) >= ord('a') and ord(x) <= ord('z')) or (ord(x) >= ord('A') and ord(x) <= ord('Z')) or (ord(x) == ord(' '))]
        d = ''.join(d)
        d = ' '.join(d.split())
        processed_data.append(d)
    return processed_data

In [58]:
joke_text_processed = cleanData(joke_text)

In [59]:
joke_text_processed[:10]

['me narrating a documentary about narrators i cant hear what theyre saying cuz im talking',
 'telling my daughter garlic is good for you EOS good immune system and keeps pests away EOS ticks',
 'ive been going through a really rough period at work this week its my own fault for swapping my tampax for sand paper EOS',
 'if i could have dinner with anyone',
 'two guys walk into a bar EOS the third guy ducks EOS',
 'why cant barbie get pregnant because ken comes in a different box EOS heyooooooo',
 'why was the musician arrested he got in treble EOS',
 'did you hear about the guy who blew his entire lottery winnings on a limousine he had nothing left to chauffeur it EOS',
 'what do you do if a bird shits on your car dont ask her out again EOS',
 'he was a real gentlemen and always opened the fridge door for me']

In [60]:
#ADD EOJ (End of joke)
for i in range(len(joke_text_processed)):
    joke_text_processed[i] = joke_text_processed[i]+" EOJ"

In [92]:
with open(cleaned_data_file,"wb") as fop:
    pickle.dump(joke_text_processed,fop)

In [61]:
joke_text_processed[:10]

['me narrating a documentary about narrators i cant hear what theyre saying cuz im talking EOJ',
 'telling my daughter garlic is good for you EOS good immune system and keeps pests away EOS ticks EOJ',
 'ive been going through a really rough period at work this week its my own fault for swapping my tampax for sand paper EOS EOJ',
 'if i could have dinner with anyone EOJ',
 'two guys walk into a bar EOS the third guy ducks EOS EOJ',
 'why cant barbie get pregnant because ken comes in a different box EOS heyooooooo EOJ',
 'why was the musician arrested he got in treble EOS EOJ',
 'did you hear about the guy who blew his entire lottery winnings on a limousine he had nothing left to chauffeur it EOS EOJ',
 'what do you do if a bird shits on your car dont ask her out again EOS EOJ',
 'he was a real gentlemen and always opened the fridge door for me EOJ']

In [64]:
#percentage
percenatge = 0.1
joke_text_processed_reduced = joke_text_processed[:int(len(joke_text_processed)*percenatge)]
joke_text_id_reduced = joke_id[:int(len(joke_id)*percenatge)]
assert len(joke_text_processed_reduced) == len(joke_text_id_reduced)
print(len(joke_text_processed)*percenatge)

23165.7


In [52]:
len(joke_text_processed_reduced)

23165

## Create the Vocabulary 

In [80]:
joke_words_list = []
for joke in joke_text_processed_reduced:
    for word in joke.split():
        joke_words_list.append(word)

In [81]:
#Num words
len(joke_words_list)

379409

In [82]:
#Num uniqe words
len(set(joke_words_list))

20650

In [93]:
# count the number of words
wordlist = joke_words_list
word_counts = collections.Counter(joke_words_list)

# Mapping from index to word : that's the vocabulary
vocabulary_inv = [x[0] for x in word_counts.most_common()]
vocabulary_inv = list(sorted(vocabulary_inv))

# Mapping from word to index
vocab = {x: i for i, x in enumerate(vocabulary_inv)}
words = [x[0] for x in word_counts.most_common()]

#size of the vocabulary
vocab_size = len(words)
print("vocab size: ", vocab_size)

#save the words and vocabulary
with open(os.path.join(vocab_file), 'wb') as f:
    pickle.dump((words, vocab, vocabulary_inv), f)


vocab size:  20650


In [98]:
#create sequences
sequences = []
next_words = []
for i in range(0, len(wordlist) - seq_length, sequences_step):
    sequences.append(wordlist[i: i + seq_length])
    next_words.append(wordlist[i + seq_length])

print('nb sequences:', len(sequences))

nb sequences: 379379
